In [1]:
import os
import tensorflow as tf 
import tensorflow_io as tfio

from datetime import datetime
from os import listdir
from os.path import isfile, join

import librosa
import librosa.display
import IPython.display as ipd

import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization, Flatten, LSTM, TimeDistributed
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint

from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
mypath = './RespiratorySoundDatabase/Respiratory_Sound_Database/Respiratory_Sound_Database/audio_and_txt_files/'
filenames = [f for f in listdir(mypath) if (isfile(join(mypath, f)) and f.endswith('.wav'))] 
print(mypath)

./RespiratorySoundDatabase/Respiratory_Sound_Database/Respiratory_Sound_Database/audio_and_txt_files/


In [3]:
p_id_in_file = [] # patient IDs corresponding to each file
for name in filenames:
    p_id_in_file.append(int(name[:3]))

p_id_in_file = np.array(p_id_in_file)
# print(p_id_in_file)

In [4]:
max_pad_len = 862 # to make the length of all MFCC equal

def extract_features(file_name):
    """
    This function takes in the path for an audio file as a string, loads it, and returns the MFCC
    of the audio"""
   
    try:
        audio, sample_rate = librosa.load(file_name, duration=20) 
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    
        pad_width = max_pad_len - mfccs.shape[1]
        mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
        
    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        return None 
    return mfccs

extract_features(rf'RespiratorySoundDatabase\Respiratory_Sound_Database\Respiratory_Sound_Database\audio_and_txt_files\104_1b1_Ll_sc_Litt3200.wav')


array([[-4.64041199e+02, -4.18290039e+02, -3.87566589e+02, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 8.44875793e+01,  1.36891495e+02,  1.71178497e+02, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 6.75648041e+01,  8.99741745e+01,  1.04197815e+02, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [-2.82355607e-01,  2.05963564e+00,  1.22030616e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-6.52661860e-01,  1.89648938e+00,  5.53389192e-01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-6.38449073e-01,  7.99845815e-01, -1.81964302e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]], dtype=float32)

In [5]:
filepaths = [join(mypath, f) for f in filenames] # full paths of files

# for f in filepaths:
#     print(f)

In [6]:
p_diag = pd.read_csv("./RespiratorySoundDatabase/Respiratory_Sound_Database/Respiratory_Sound_Database/patient_diagnosis.csv",header=None) # patient diagnosis file
print(p_diag)

       0          1
0    101       URTI
1    102    Healthy
2    103       COPD
3    104       COPD
4    105       URTI
..   ...        ...
121  222       COPD
122  223       COPD
123  224    Healthy
124  225    Healthy
125  226  Pneumonia

[126 rows x 2 columns]


In [8]:
# labels = np.array([p_diag[p_diag[0] == x][1].values[0] for x in p_id_in_file]) # labels for audio files

labels = [p_diag[p_diag[0] == x][1].values[0] for x in p_id_in_file]
print(type(labels))
print(len(labels))
# print(labels)

<class 'list'>
920


In [9]:
from soundfile import SoundFile
features = []
idx = 0 
# Iterate through each sound file and extract the features
for file_name in filepaths:
    with SoundFile(file_name, 'r') as file_name:
        data = extract_features(file_name)
        features.append(data)

print('Finished feature extraction from ', len(features), ' files')
# features = np.array(features,dtype=object).flatten() 
# for i in features:
#     print(i)

Finished feature extraction from  920  files


In [9]:
print(len(features))

920


In [10]:
X = features
y = labels
print(type(X))
print(type(y))
X = np.array(X)
y = np.array(y)
print(type(X))
print(type(y))

<class 'list'>
<class 'list'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [123]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(len(X_train))
print(len(y_train))
print(len(X_test))
print(len(y_test))
print(len(X_train[0]))

from keras.utils import to_categorical

# One-hot encode the training labels
y_train = to_categorical(y_train)
# print(y_train)
y_test = to_categorical(y_test)

736
736
184
184
40


In [124]:
num_rows = 40
num_columns = 862
num_channels = 1
model = Sequential()
model.add(Conv2D(64, kernel_size=5, strides=1, padding='same', input_shape=(num_rows, num_columns, num_channels), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128, kernel_size=5, strides=1, padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(6, activation='softmax'))


In [125]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam') 

In [126]:
model.summary()

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_39 (Conv2D)          (None, 40, 862, 64)       1664      
                                                                 
 batch_normalization_38 (Bat  (None, 40, 862, 64)      256       
 chNormalization)                                                
                                                                 
 max_pooling2d_38 (MaxPoolin  (None, 20, 431, 64)      0         
 g2D)                                                            
                                                                 
 conv2d_40 (Conv2D)          (None, 20, 431, 128)      204928    
                                                                 
 batch_normalization_39 (Bat  (None, 20, 431, 128)     512       
 chNormalization)                                                
                                                     

In [127]:

model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
23/23 [==============================] - 58s 2s/step - loss: 57.8647 - accuracy: 0.6630 - val_loss: 115.1548 - val_accuracy: 0.8152
Epoch 2/10
23/23 [==============================] - 55s 2s/step - loss: 9.2784 - accuracy: 0.7867 - val_loss: 46.1118 - val_accuracy: 0.1685
Epoch 3/10
23/23 [==============================] - 56s 2s/step - loss: 1.9915 - accuracy: 0.8207 - val_loss: 13.4288 - val_accuracy: 0.7609
Epoch 4/10
23/23 [==============================] - 57s 2s/step - loss: 0.7870 - accuracy: 0.8397 - val_loss: 6.8191 - val_accuracy: 0.7283
Epoch 5/10
23/23 [==============================] - 56s 2s/step - loss: 0.4607 - accuracy: 0.8777 - val_loss: 3.1067 - val_accuracy: 0.7554
Epoch 6/10
23/23 [==============================] - 56s 2s/step - loss: 0.5041 - accuracy: 0.8750 - val_loss: 3.5673 - val_accuracy: 0.7989
Epoch 7/10
23/23 [==============================] - 57s 2s/step - loss: 0.4075 - accuracy: 0.8764 - val_loss: 2.5712 - val_accuracy: 0.8098
Epoch 8/10
23/2

In [128]:
# Evaluating the model on the training and testing set
score = model.evaluate(X_train, y_train)
print("Training Accuracy: ", score[1])

score = model.evaluate(X_test, y_test)
print("Testing Accuracy: ", score[1])

23/23 [==============================] - 10s 444ms/step - loss: 0.4561 - accuracy: 0.8818
Training Accuracy:  0.8817934989929199
6/6 [==============================] - 3s 417ms/step - loss: 0.6234 - accuracy: 0.8152
Testing Accuracy:  0.8152173757553101


In [129]:
model.save('my_model.h5')

In [200]:
model = tf.keras.models.load_model('my_model.h5')

# Load the input data
input_data = extract_features('./RespiratorySoundDatabase/Respiratory_Sound_Database/Respiratory_Sound_Database/audio_and_txt_files/104_1b1_Ll_sc_Litt3200.wav')


input_data = input_data.reshape(1, 40, 862, 1)
print(input_data.shape)




(1, 40, 862, 1)


In [201]:

lbls = np.unique(labels)
print(lbls)
print(filepaths[2])
print(labels[2])

['Bronchiectasis' 'Bronchiolitis' 'COPD' 'Healthy' 'Pneumonia' 'URTI']
./RespiratorySoundDatabase/Respiratory_Sound_Database/Respiratory_Sound_Database/audio_and_txt_files/102_1b1_Ar_sc_Meditron.wav
Healthy


In [205]:
# Make predictions
predictions = model.predict(input_data)

# Print the predictions
predictions = predictions.reshape(6)
print(predictions)
mx = predictions[0]
index = 0 
for i in predictions:
    if i > mx :
        mx = i 
# print(mx)
indices = np.where(predictions == mx)

# Print the indices
# print(indices)

# Create the dictionary
dictionary = dict(zip(lbls, predictions))

# print(dictionary)
max_element = max(dictionary,key=dictionary.get)
print(max_element)

1/1 [==============================] - 0s 69ms/step
[0.01439044 0.03732305 0.6423005  0.09150169 0.13805687 0.07642743]
COPD
